In [1]:
import sys
import os
sys.path.insert(1, '/braintree/home/andrewci/brain-score/')
sys.path.insert(1, '/braintree/home/andrewci/brain-score/packaging/dicarlo/cirincione/VOneNet_DN_Code')
from candidate_models.model_commitments import brain_translated_pool
from cirincione2022_zhou2000_TESTING import load_model_and_BO_stimulus, BO_optimization_and_standard_test, create_response_ratio_plots, load_model_and_BO_stimulus_LAYERS, V1_LAYERS, region_accuracy, run_v1block_DN
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

/braintree/home/andrewci/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.8' currently installed).
  warnings.warn(msg, UserWarning)
/braintree/home/andrewci/anaconda3/lib/python3.7/site-packages/dask/config.py:161: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/braintree/home/andrewci/anaconda3/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)
/braintree/home/andrewci/brain-score/brainscore/metrics/__init__.py:37: FutureWarning: xarray subclass Score should explicitly define __slots__
  class Score(DataAssembly):


In [2]:
optim_test_dir = '/braintree/data2/active/users/andrewci/stimulus_sets/dicarlo.Cirincione2022_border_ownership_optimization_test'
standard_test_dir = '/braintree/data2/active/users/andrewci/stimulus_sets/dicarlo.Cirincione2022_border_ownership_standard_test'
BO_STIM_NAME = 'dicarlo.Cirincione2022_border_ownership_standard_test'
BO_OPTIM_NAME = 'dicarlo.Cirincione2022_border_ownership_optimization_test'

In [3]:
os.environ['CUDA_VISIBLE_DEVICES']= '3'
model = run_v1block_DN()
model_identifier = 'VOneNet_DN'
region = 'V1'
model, optimization_test_responses, standard_test_responses = load_model_and_BO_stimulus(BO_OPTIM_NAME=BO_OPTIM_NAME, optim_test_dir=optim_test_dir,
                    BO_STIM_NAME=BO_STIM_NAME, standard_test_dir=standard_test_dir, model_identifier=model_identifier, region=region, model=model)
save_path = '/braintree/data2/active/users/andrewci/BO_Responses/'
BO_responses = BO_optimization_and_standard_test(optimization_test_responses=optimization_test_responses, standard_test_responses=standard_test_responses,
model_identifier = model_identifier, path=save_path, save_figs_and_data=1)
create_response_ratio_plots(model_identifier=model_identifier, BO_responses=BO_responses, 
                    region=region, save_figs_and_data=1, path=save_path)

VOne Net: True
Divisive Normalization: True
Color Channels: 3
Visual Degrees: 8
Restore Path is: None
Map location is: cuda
Full imagenet: False
Image size:256
Constructing VOneNet
Neuronal distributions gabor parameters
Stride is 2
In channels is 3


/braintree/home/andrewci/brain-score/packaging/dicarlo/cirincione/VOneNet_DN_Code/params.py:60: RuntimeWarning: invalid value encountered in true_divide
  ny_dist_marg = n_joint_dist / n_joint_dist.sum(axis=1, keepdims=True)
/braintree/home/andrewci/anaconda3/lib/python3.7/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Model:  VOneResnet18
Model:  VOneNet DN


/braintree/home/andrewci/anaconda3/lib/python3.7/site-packages/torch/nn/parallel/data_parallel.py:30: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 4 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))


Loading catalog from entrypoints
Loading lookup from /braintree/home/andrewci/brain-score/brainscore/lookup.csv


KeyboardInterrupt: 

In [ ]:
#all_models=list(brain_translated_pool.keys())
#print(all_models)
# alexnet
# cornets
# resnet-50-pytorch
# resnet-50-robust

In [ ]:
#select_models = ['resnet-50-pytorch', 'resnet-50-robust', 'CORnet-S', 'alexnet', 'voneresnet-50-non_stochastic']
select_models = ['resnet-50-pytorch']

In [ ]:

for i in tqdm(select_models):
    model_identifier = i
    layer_count = V1_LAYERS[model_identifier]
    V1_similarity_vec = np.zeros(len(layer_count))
    V2_similarity_vec = np.zeros(len(layer_count))
    for L in tqdm(layer_count):
        region = 'V1'
        model, optimization_test_responses, standard_test_responses = load_model_and_BO_stimulus_LAYERS(BO_OPTIM_NAME=BO_OPTIM_NAME, optim_test_dir=optim_test_dir,
                    BO_STIM_NAME=BO_STIM_NAME, standard_test_dir=standard_test_dir, model_identifier=model_identifier, region=region, layer_index = L)
        save_path = '/braintree/data2/active/users/andrewci/BO_Responses/'
        BO_responses = BO_optimization_and_standard_test(optimization_test_responses=optimization_test_responses, standard_test_responses=standard_test_responses,
                    model_identifier=model_identifier, path=save_path, save_figs_and_data=1, layer_index = L)
        V1_similarity_vec[L], V2_similarity_vec[L] = create_response_ratio_plots(model_identifier=model_identifier, BO_responses=BO_responses, 
                    region=region, save_figs_and_data=1, path=save_path, layer_index = L)
        print(f'V1 score is {V1_similarity_vec[L]}')
        print(f'V2 score is {V2_similarity_vec[L]}')
    region_accuracy(V1_similarity_vec, V2_similarity_vec, model_identifier, save_path)
        

In [ ]:
#median response ratio